## Process the SST data

Their units are °C 

In [2]:
# Import necessary packages

import xarray as xr

In [33]:
# Import the files

hadisst = xr.open_dataset('/Users/iriskeizer/Documents/Wind effect/Data/observations/SST/HadISST_sst.nc')
cobesst = xr.open_dataset('/Users/iriskeizer/Documents/Wind effect/Data/observations/SST/cobe_sst.mon.mean.nc')

# Obtain annual averages
hadisst = hadisst.groupby('time.year').mean('time')
cobesst = cobesst.groupby('time.year').mean('time')

# Delete 2022
hadisst = hadisst.where((hadisst.year<2022), drop=True)
cobesst = cobesst.where((cobesst.year<2022), drop=True)

# Shift longitudes to -180-180
cobesst.coords['lon'] = (cobesst.lon + 180) % 360 - 180

# Rename coordinates
hadisst = hadisst.rename({'latitude': 'lat', 'longitude': 'lon', 'year': 'time'})
cobesst = cobesst.rename({'year': 'time'})


# Sort dataarrays ascending
hadisst = hadisst.sortby(hadisst.lon, ascending=True)
hadisst = hadisst.sortby(hadisst.lat, ascending=True)
cobesst = cobesst.sortby(cobesst.lon, ascending=True)
cobesst = cobesst.sortby(cobesst.lat, ascending=True)



def xr_select_region(data, coords):
    '''
    Function that selects a given region of data for xarray dataset  
    
    Coords should have shape: [longitude minimum, longitude maximum, latitude minimum, latitude maximum]
    '''
    
    data = data.where(data.lon>=coords[0], drop=True)
    data = data.where(data.lon<=coords[1], drop=True)
    data = data.where(data.lat>=coords[2], drop=True)
    data = data.where(data.lat<=coords[3], drop=True)
    
    return data



# Select North Atlantic region
hadisst = xr_select_region(hadisst, [-100, 10, 0, 90])
cobesst = xr_select_region(cobesst, [-100, 10, 0, 90])


# Save datasets
hadisst.sst.to_netcdf('/Users/iriskeizer/Documents/Wind effect/Data/observations/SST/hadisst_processed.nc')
cobesst.sst.to_netcdf('/Users/iriskeizer/Documents/Wind effect/Data/observations/SST/cobesst_processed.nc')

